# 🧠 Gianna Semantic Memory System - Complete Demo

This notebook demonstrates the comprehensive semantic memory system implemented for Gianna AI Assistant, including:

- **Embedding-based storage** with multiple provider support
- **Vector similarity search** using Chroma, FAISS, or in-memory stores
- **Automatic clustering** of similar interactions
- **Context summarization** and pattern detection
- **Integration with GiannaState** management
- **Tool integration** for LangGraph workflows

## Features Demonstrated

✅ **Multi-provider embeddings** (OpenAI, SentenceTransformers, HuggingFace)
✅ **Multiple vector stores** (Chroma, FAISS, In-Memory)
✅ **Semantic similarity search** with configurable thresholds
✅ **Automatic clustering** of similar conversations
✅ **Pattern detection** and user preference analysis
✅ **Context summarization** with intelligent aggregation
✅ **Fallback strategies** when advanced providers aren't available
✅ **Integration with existing GiannaState** system
✅ **Tool interface** for AI agents and workflows

## 🔧 Setup and Dependencies

Let's start by checking available dependencies and setting up the memory system.

In [ ]:
# Check for optional dependencies
dependencies_status = {
    "numpy": False,
    "sentence_transformers": False,
    "transformers": False,
    "torch": False,
    "faiss": False,
    "chromadb": False,
    "openai": False
}

for package in dependencies_status:
    try:
        __import__(package)
        dependencies_status[package] = True
        print(f"✅ {package} is available")
    except ImportError:
        print(f"❌ {package} is not available")

print("\n📊 Dependency Status Summary:")
available = sum(dependencies_status.values())
total = len(dependencies_status)
print(f"Available: {available}/{total} ({(available/total)*100:.1f}%)")

In [ ]:
# Core imports
import sys
import os
import json
from datetime import datetime, timedelta
from pathlib import Path

# Add gianna to path if needed
if '../' not in sys.path:
    sys.path.append('../')

# Import Gianna components
try:
    from gianna.memory import SemanticMemory, MemoryConfig
    from gianna.memory.embeddings import EmbeddingProvider, get_available_providers
    from gianna.memory.vectorstore import VectorStoreProvider, get_available_vector_stores
    from gianna.memory.state_integration import MemoryIntegratedStateManager
    from gianna.tools.memory_tools import create_semantic_memory_tool, get_available_memory_tools
    
    GIANNA_MEMORY_AVAILABLE = True
    print("✅ Gianna semantic memory system imported successfully")
except ImportError as e:
    print(f"❌ Failed to import Gianna memory system: {e}")
    GIANNA_MEMORY_AVAILABLE = False

## 🚀 Basic Memory System Demo

Let's start with a basic demonstration of storing and retrieving conversations using semantic similarity.

In [ ]:
if not GIANNA_MEMORY_AVAILABLE:
    print("⚠️ Skipping demos - Gianna memory system not available")
else:
    # Check available providers
    available_embeddings = get_available_providers()
    available_vectorstores = get_available_vector_stores()
    available_tools = get_available_memory_tools()
    
    print("🔍 Available Providers:")
    print(f"  Embeddings: {[p.value for p in available_embeddings]}")
    print(f"  Vector Stores: {[p.value for p in available_vectorstores]}")
    print(f"  Memory Tools: {available_tools}")
    
    # Create configuration with best available options
    if available_embeddings:
        embedding_provider = available_embeddings[0]  # Use first available
    else:
        print("❌ No embedding providers available")
        embedding_provider = None
    
    if available_vectorstores:
        vectorstore_provider = available_vectorstores[0] if VectorStoreProvider.CHROMA in available_vectorstores else available_vectorstores[0]
    else:
        vectorstore_provider = VectorStoreProvider.IN_MEMORY  # Fallback
    
    print(f"\n🔧 Using: {embedding_provider.value if embedding_provider else 'None'} + {vectorstore_provider.value}")

In [ ]:
if GIANNA_MEMORY_AVAILABLE and embedding_provider:
    # Create memory configuration
    config = MemoryConfig(
        embedding_provider=embedding_provider,
        vectorstore_provider=vectorstore_provider,
        similarity_threshold=0.7,
        max_search_results=5,
        enable_clustering=True,
        cluster_similarity_threshold=0.85
    )
    
    # Initialize semantic memory
    try:
        memory = SemanticMemory(config)
        print("✅ Semantic memory initialized successfully")
        print(f"   Configuration: {config.embedding_provider.value} + {config.vectorstore_provider.value}")
        print(f"   Similarity threshold: {config.similarity_threshold}")
        print(f"   Clustering enabled: {config.enable_clustering}")
    except Exception as e:
        print(f"❌ Failed to initialize semantic memory: {e}")
        memory = None
else:
    print("⚠️ Skipping memory initialization - requirements not met")
    memory = None

### 📝 Storing Sample Interactions

Let's store some diverse interactions to demonstrate the memory system capabilities.

In [ ]:
if memory:
    # Sample interactions covering different topics
    sample_interactions = [
        # Programming and AI cluster
        {
            "user_input": "How do I implement a neural network in Python?",
            "assistant_response": "You can implement a neural network in Python using frameworks like TensorFlow or PyTorch. Start with defining layers, activation functions, and training loops.",
            "context": "AI/ML programming discussion",
            "type": "programming",
            "topic": "neural_networks"
        },
        {
            "user_input": "What's the difference between supervised and unsupervised learning?",
            "assistant_response": "Supervised learning uses labeled training data to learn input-output mappings, while unsupervised learning finds patterns in data without labels.",
            "context": "Machine learning concepts",
            "type": "education", 
            "topic": "machine_learning"
        },
        {
            "user_input": "Can you help me debug this Python code?",
            "assistant_response": "I'd be happy to help debug your Python code! Please share the code and describe the error you're encountering.",
            "context": "Programming assistance",
            "type": "programming",
            "topic": "debugging"
        },
        
        # Web development cluster
        {
            "user_input": "How do I create a REST API with Flask?",
            "assistant_response": "To create a REST API with Flask: 1) Install Flask, 2) Create app instance, 3) Define routes with decorators, 4) Handle HTTP methods, 5) Return JSON responses.",
            "context": "Web development tutorial",
            "type": "programming",
            "topic": "web_development"
        },
        {
            "user_input": "What are the best practices for API design?",
            "assistant_response": "API design best practices include: use RESTful conventions, proper HTTP status codes, consistent naming, versioning, authentication, rate limiting, and comprehensive documentation.",
            "context": "Software architecture discussion",
            "type": "architecture",
            "topic": "api_design"
        },
        
        # Data science cluster
        {
            "user_input": "How do I clean messy data in pandas?",
            "assistant_response": "Data cleaning in pandas involves: handling missing values with dropna()/fillna(), removing duplicates with drop_duplicates(), data type conversion with astype(), and outlier detection.",
            "context": "Data preprocessing tutorial",
            "type": "data_science",
            "topic": "data_cleaning"
        },
        {
            "user_input": "What's the best way to visualize time series data?",
            "assistant_response": "For time series visualization, use line plots for trends, seasonal decomposition plots for patterns, and interactive plots with plotly for exploration. Consider matplotlib, seaborn, or plotly.",
            "context": "Data visualization guidance",
            "type": "data_science",
            "topic": "visualization"
        },
        
        # General assistance cluster
        {
            "user_input": "Can you recommend some good tech books?",
            "assistant_response": "Great tech books include: 'Clean Code' by Robert Martin, 'The Pragmatic Programmer' by Hunt & Thomas, 'Designing Data-Intensive Applications' by Martin Kleppmann.",
            "context": "Book recommendations",
            "type": "recommendations",
            "topic": "books"
        },
        {
            "user_input": "What's the weather like today?",
            "assistant_response": "I don't have access to real-time weather data. Please check a weather app or website like weather.com for current conditions in your area.",
            "context": "General inquiry",
            "type": "general",
            "topic": "weather"
        }
    ]
    
    print(f"📝 Storing {len(sample_interactions)} sample interactions...")
    session_id = "demo_session_001"
    interaction_ids = []
    
    for i, interaction in enumerate(sample_interactions, 1):
        interaction_id = memory.store_interaction(
            session_id=session_id,
            user_input=interaction["user_input"],
            assistant_response=interaction["assistant_response"],
            context=interaction["context"],
            interaction_type=interaction["type"],
            metadata={
                "topic": interaction["topic"],
                "demo_sequence": i,
                "timestamp": datetime.now().isoformat()
            }
        )
        
        if interaction_id:
            interaction_ids.append(interaction_id)
            print(f"  ✅ [{i:2d}] {interaction['type']:12} | {interaction['user_input'][:50]}...")
        else:
            print(f"  ❌ [{i:2d}] Failed to store interaction")
    
    print(f"\n✅ Successfully stored {len(interaction_ids)} interactions")
    
else:
    print("⚠️ Skipping interaction storage - memory system not initialized")

### 🔍 Semantic Similarity Search

Now let's demonstrate the power of semantic search - finding related conversations even with different wording.

In [ ]:
if memory:
    print("🔍 Semantic Search Demonstrations\n")
    
    # Test queries with semantic similarity
    search_queries = [
        {
            "query": "machine learning and artificial intelligence",
            "description": "Should find neural networks, ML concepts"
        },
        {
            "query": "building web applications and servers", 
            "description": "Should find Flask API, web development"
        },
        {
            "query": "data analysis and processing techniques",
            "description": "Should find pandas, data visualization"
        },
        {
            "query": "code issues and troubleshooting",
            "description": "Should find debugging assistance"
        },
        {
            "query": "learning resources and educational materials",
            "description": "Should find book recommendations"
        }
    ]
    
    for i, search in enumerate(search_queries, 1):
        print(f"🔎 Search {i}: '{search['query']}'")
        print(f"   Expected: {search['description']}")
        
        try:
            results = memory.search_similar_interactions(
                query=search["query"],
                session_id=session_id,
                max_results=3,
                similarity_threshold=0.5  # Lower threshold for demo
            )
            
            if results:
                print(f"   Found {len(results)} similar interactions:")
                for j, result in enumerate(results, 1):
                    topic = result.metadata.get('topic', 'unknown')
                    print(f"     {j}. [{topic:15}] {result.user_input[:60]}...")
            else:
                print("     No similar interactions found")
                
        except Exception as e:
            print(f"     ❌ Search error: {e}")
        
        print()  # Empty line for readability
        
else:
    print("⚠️ Skipping semantic search demo - memory system not initialized")

### 🧩 Pattern Detection and Analysis

The system can automatically detect patterns in user interactions, including topics, preferences, and conversation patterns.

In [ ]:
if memory:
    print("🧩 Pattern Detection and Analysis\n")
    
    try:
        patterns = memory.detect_patterns(session_id)
        
        print("📊 Interaction Statistics:")
        print(f"   Total interactions: {patterns.get('total_interactions', 0)}")
        
        if 'time_span' in patterns:
            time_span = patterns['time_span']
            print(f"   Session duration: {time_span.get('duration_minutes', 0)} minutes")
        
        print("\n🏷️ Interaction Types:")
        for interaction_type, count in patterns.get('interaction_types', {}).items():
            print(f"   {interaction_type:15}: {count} interactions")
        
        print("\n💭 Common Themes:")
        themes = patterns.get('common_themes', [])[:8]  # Show top 8
        if themes:
            for theme in themes:
                print(f"   • {theme}")
        else:
            print("   No common themes detected")
        
        print("\n👤 User Preferences:")
        preferences = patterns.get('user_preferences', {})
        for key, value in preferences.items():
            if isinstance(value, (int, float)):
                print(f"   {key:25}: {value:.1f}")
            else:
                print(f"   {key:25}: {value}")
        
        print("\n🗣️ Conversation Patterns:")
        conv_patterns = patterns.get('conversation_patterns', {})
        for key, value in conv_patterns.items():
            if isinstance(value, float):
                print(f"   {key:25}: {value:.3f}")
            else:
                print(f"   {key:25}: {value}")
        
        print("\n🎯 Detected Clusters:")
        clusters = patterns.get('clusters', {})
        if clusters:
            for i, (cluster_id, interaction_ids) in enumerate(clusters.items(), 1):
                print(f"   Cluster {i}: {len(interaction_ids)} interactions (ID: {cluster_id[:8]}...)")
        else:
            print("   No clusters detected (need more similar interactions)")
            
    except Exception as e:
        print(f"❌ Pattern detection error: {e}")

else:
    print("⚠️ Skipping pattern analysis - memory system not initialized")

### 📋 Context Summarization

The system can generate intelligent summaries of conversation context, useful for maintaining context across long conversations.

In [ ]:
if memory:
    print("📋 Context Summarization Demo\n")
    
    try:
        # Get context summary for different interaction counts
        summary_configs = [
            {"max_interactions": 3, "description": "Recent 3 interactions"},
            {"max_interactions": 5, "description": "Recent 5 interactions"},
            {"max_interactions": None, "description": "All interactions (default window)"}
        ]
        
        for config in summary_configs:
            print(f"📄 {config['description']}:")
            print("-" * 50)
            
            summary = memory.get_context_summary(
                session_id=session_id,
                max_interactions=config['max_interactions']
            )
            
            # Format the summary nicely
            lines = summary.split('\n')
            for line in lines:
                if line.strip():
                    print(f"   {line}")
            
            print()  # Empty line between summaries
            
    except Exception as e:
        print(f"❌ Context summary error: {e}")

else:
    print("⚠️ Skipping context summarization - memory system not initialized")

### 📈 Memory System Statistics

Let's examine the memory system performance and statistics.

In [ ]:
if memory:
    print("📈 Memory System Statistics\n")
    
    try:
        stats = memory.get_memory_stats()
        
        print("🏗️ System Configuration:")
        print(f"   Embedding provider:        {stats.get('embedding_provider')}")
        print(f"   Vector store provider:      {stats.get('vector_store_provider')}")
        print(f"   Embedding dimension:        {stats.get('embedding_dimension')}")
        print(f"   Similarity threshold:       {stats.get('average_similarity_threshold')}")
        
        print("\n📊 Storage Statistics:")
        print(f"   Total interactions:         {stats.get('total_interactions')}")
        print(f"   Cached interactions:        {stats.get('cached_interactions')}")
        print(f"   Total summaries:            {stats.get('total_summaries')}")
        
        print("\n⚡ Performance Metrics:")
        print(f"   Cache hit rate:             {stats.get('cache_hit_rate', 0):.1%}")
        
    except Exception as e:
        print(f"❌ Statistics error: {e}")

else:
    print("⚠️ Skipping statistics - memory system not initialized")

## 🚀 Advanced Features Demo

Let's explore some advanced features including filtered search, global search across sessions, and memory cleanup.

In [ ]:
if memory:
    print("🔍 Advanced Search Features\n")
    
    # Store interactions in a second session for global search demo
    second_session_id = "demo_session_002"
    
    # Add some interactions to second session
    additional_interactions = [
        {
            "user_input": "How do I deploy a machine learning model to production?",
            "assistant_response": "ML model deployment involves containerization with Docker, API creation, monitoring, scaling, and CI/CD pipelines. Consider platforms like AWS SageMaker or Kubernetes.",
            "context": "MLOps discussion",
            "type": "deployment"
        },
        {
            "user_input": "What are the security considerations for web APIs?",
            "assistant_response": "Web API security includes authentication (JWT/OAuth), authorization, input validation, rate limiting, HTTPS encryption, and protection against common attacks like CSRF and injection.",
            "context": "Security best practices",
            "type": "security"
        }
    ]
    
    print(f"📝 Adding {len(additional_interactions)} interactions to second session...")
    for interaction in additional_interactions:
        memory.store_interaction(
            session_id=second_session_id,
            user_input=interaction["user_input"],
            assistant_response=interaction["assistant_response"],
            context=interaction["context"],
            interaction_type=interaction["type"],
            metadata={"session": "second"}
        )
    
    print("\n🔎 Filtered Search by Interaction Type:")
    programming_results = memory.search_similar_interactions(
        query="coding and development",
        session_id=session_id,
        interaction_type="programming",
        max_results=3
    )
    
    print(f"   Found {len(programming_results)} programming-related interactions:")
    for result in programming_results:
        print(f"     • {result.user_input[:60]}...")
    
    print("\n🌍 Global Search Across All Sessions:")
    global_results = memory.search_similar_interactions(
        query="machine learning and AI models",
        session_id=None,  # Search all sessions
        max_results=5
    )
    
    print(f"   Found {len(global_results)} interactions across all sessions:")
    for result in global_results:
        session_label = "Session 1" if result.session_id == session_id else "Session 2"
        print(f"     • [{session_label}] {result.user_input[:50]}...")

else:
    print("⚠️ Skipping advanced search features - memory system not initialized")

## 🔧 Tool Integration Demo

The semantic memory system integrates with Gianna's tool system for use in AI agents and workflows.

In [ ]:
if memory and "semantic_memory" in get_available_memory_tools():
    print("🔧 Tool Integration Demo\n")
    
    # Create semantic memory tool
    try:
        memory_tool = create_semantic_memory_tool(config)
        
        if memory_tool:
            print("✅ Semantic memory tool created successfully")
            print(f"   Tool name: {memory_tool.name}")
            print(f"   Description: {memory_tool.description[:80]}...")
            
            # Test tool operations
            print("\n🧪 Testing Tool Operations:")
            
            # Test search operation
            search_request = {
                "action": "search",
                "query": "Python programming and development",
                "session_id": session_id,
                "max_results": 2,
                "similarity_threshold": 0.6
            }
            
            print(f"\n🔍 Search Request: {search_request['query']}")
            search_result = memory_tool._run(json.dumps(search_request))
            search_data = json.loads(search_result)
            
            if search_data.get('success'):
                results = search_data.get('results', [])
                print(f"   ✅ Found {len(results)} results")
                for i, result in enumerate(results, 1):
                    print(f"     {i}. {result['user_input'][:50]}...")
            else:
                print(f"   ❌ Search failed: {search_data.get('error')}")
            
            # Test pattern detection
            pattern_request = {
                "action": "patterns",
                "session_id": session_id
            }
            
            print(f"\n🧩 Pattern Detection Request")
            pattern_result = memory_tool._run(json.dumps(pattern_request))
            pattern_data = json.loads(pattern_result)
            
            if pattern_data.get('success'):
                patterns = pattern_data.get('patterns', {})
                print(f"   ✅ Pattern analysis completed")
                print(f"   Total interactions: {patterns.get('total_interactions', 0)}")
                print(f"   Interaction types: {list(patterns.get('interaction_types', {}).keys())}")
            else:
                print(f"   ❌ Pattern detection failed: {pattern_data.get('error')}")
            
            # Test statistics
            stats_request = {"action": "stats"}
            
            print(f"\n📈 Statistics Request")
            stats_result = memory_tool._run(json.dumps(stats_request))
            stats_data = json.loads(stats_result)
            
            if stats_data.get('success'):
                stats = stats_data.get('stats', {})
                print(f"   ✅ Statistics retrieved")
                print(f"   Provider: {stats.get('embedding_provider')} + {stats.get('vector_store_provider')}")
                print(f"   Total interactions: {stats.get('total_interactions')}")
            else:
                print(f"   ❌ Statistics failed: {stats_data.get('error')}")
        
        else:
            print("❌ Failed to create semantic memory tool")
            
    except Exception as e:
        print(f"❌ Tool integration error: {e}")

else:
    print("⚠️ Skipping tool integration demo - semantic memory tool not available")

## 🔄 State Manager Integration

The semantic memory system integrates seamlessly with Gianna's existing state management system.

In [ ]:
if GIANNA_MEMORY_AVAILABLE:
    print("🔄 State Manager Integration Demo\n")
    
    try:
        from gianna.core.state import create_initial_state
        
        # Create memory-integrated state manager
        integrated_manager = MemoryIntegratedStateManager(
            db_path="demo_gianna_state.db",
            memory_config=config
        )
        
        print("✅ Memory-integrated state manager created")
        
        # Create a demo state with conversation
        test_session_id = "state_integration_test"
        state = create_initial_state(test_session_id)
        
        # Add some conversation messages
        state["conversation"].messages = [
            {
                "role": "user",
                "content": "How do I optimize database queries?",
                "timestamp": datetime.now().isoformat()
            },
            {
                "role": "assistant", 
                "content": "Database query optimization involves indexing, query analysis, proper joins, avoiding N+1 problems, and using database-specific optimization features.",
                "timestamp": datetime.now().isoformat()
            }
        ]
        
        print("\n💾 Saving state with automatic semantic memory integration...")
        integrated_manager.save_state(test_session_id, state)
        print("   ✅ State saved (conversations automatically stored in semantic memory)")
        
        # Load state with enhanced context
        print("\n📖 Loading state with enhanced semantic context...")
        loaded_state = integrated_manager.load_state(test_session_id)
        
        if loaded_state:
            print("   ✅ State loaded successfully")
            print(f"   Messages: {len(loaded_state['conversation'].messages)}")
            if loaded_state["conversation"].context_summary:
                print(f"   Enhanced context available: {len(loaded_state['conversation'].context_summary)} characters")
        
        # Test semantic search through integrated manager
        print("\n🔍 Testing semantic search through integrated manager...")
        similar_convs = integrated_manager.search_similar_conversations(
            query="database and SQL optimization",
            session_id=test_session_id,
            max_results=3
        )
        
        print(f"   Found {len(similar_convs)} similar conversations")
        for conv in similar_convs:
            print(f"     • {conv.user_input[:50]}...")
        
        # Get comprehensive statistics
        print("\n📊 Comprehensive Memory Statistics:")
        comprehensive_stats = integrated_manager.get_memory_statistics()
        for key, value in comprehensive_stats.items():
            print(f"   {key:25}: {value}")
        
    except Exception as e:
        print(f"❌ State integration error: {e}")
        import traceback
        traceback.print_exc()

else:
    print("⚠️ Skipping state integration demo - Gianna memory system not available")

## ⚡ Performance and Scalability

Let's test the system's performance with a larger dataset and examine scalability characteristics.

In [ ]:
if memory:
    import time
    
    print("⚡ Performance and Scalability Testing\n")
    
    # Performance test with batch operations
    performance_session = "performance_test_session"
    
    # Generate more test interactions for performance testing
    performance_interactions = [
        {
            "user_input": f"What is the best approach for {topic}?",
            "assistant_response": f"For {topic}, I recommend following best practices including proper planning, testing, and documentation.",
            "context": f"Discussion about {topic}",
            "type": "question",
            "topic": topic
        }
        for topic in [
            "microservices architecture", "containerization with Docker", 
            "continuous integration", "test-driven development",
            "database design patterns", "API rate limiting",
            "caching strategies", "load balancing",
            "monitoring and observability", "security hardening",
            "performance optimization", "code review processes",
            "error handling patterns", "logging best practices",
            "configuration management"
        ]
    ]
    
    print(f"🏃 Performance Test: Storing {len(performance_interactions)} interactions...")
    start_time = time.time()
    
    stored_count = 0
    for interaction in performance_interactions:
        interaction_id = memory.store_interaction(
            session_id=performance_session,
            user_input=interaction["user_input"],
            assistant_response=interaction["assistant_response"],
            context=interaction["context"],
            interaction_type=interaction["type"],
            metadata={"topic": interaction["topic"], "batch": "performance_test"}
        )
        if interaction_id:
            stored_count += 1
    
    storage_time = time.time() - start_time
    
    print(f"   ✅ Stored {stored_count} interactions in {storage_time:.2f} seconds")
    print(f"   Average storage time: {(storage_time/stored_count)*1000:.1f} ms per interaction")
    
    # Test search performance
    search_queries = [
        "software architecture and design patterns",
        "DevOps and deployment strategies", 
        "performance monitoring and optimization",
        "security and best practices"
    ]
    
    print(f"\n🔍 Search Performance Test: {len(search_queries)} queries...")
    total_search_time = 0
    total_results = 0
    
    for query in search_queries:
        start_time = time.time()
        results = memory.search_similar_interactions(
            query=query,
            session_id=performance_session,
            max_results=5
        )
        search_time = time.time() - start_time
        total_search_time += search_time
        total_results += len(results)
        
        print(f"   Query: '{query[:40]}...' → {len(results)} results in {search_time*1000:.1f} ms")
    
    print(f"\n📊 Search Performance Summary:")
    print(f"   Average search time: {(total_search_time/len(search_queries))*1000:.1f} ms")
    print(f"   Average results per query: {total_results/len(search_queries):.1f}")
    
    # Final system statistics
    print("\n📈 Final System Statistics:")
    final_stats = memory.get_memory_stats()
    print(f"   Total interactions: {final_stats.get('total_interactions')}")
    print(f"   Cached interactions: {final_stats.get('cached_interactions')}")
    print(f"   Cache efficiency: {final_stats.get('cache_hit_rate', 0):.1%}")

else:
    print("⚠️ Skipping performance testing - memory system not initialized")

## 🧹 Memory Cleanup and Maintenance

The system includes automatic cleanup capabilities to manage storage over time.

In [ ]:
if memory:
    print("🧹 Memory Cleanup and Maintenance\n")
    
    # Show current statistics before cleanup
    print("📊 Before Cleanup:")
    stats_before = memory.get_memory_stats()
    print(f"   Total interactions: {stats_before.get('total_interactions')}")
    print(f"   Cached interactions: {stats_before.get('cached_interactions')}")
    
    # Simulate cleanup (we'll use a very small age threshold for demo)
    print("\n🗑️ Simulating cleanup of very old interactions...")
    print("   Note: Using 0 days threshold for demo - in practice use 30+ days")
    
    try:
        # In real usage, you'd use a reasonable threshold like 30 days
        # For demo purposes, we'll show what the cleanup would report
        cleanup_count = memory.cleanup_old_interactions(max_age_days=0)
        print(f"   Cleaned up {cleanup_count} interactions")
        
        if cleanup_count == 0:
            print("   (No interactions old enough to clean up in this demo)")
        
    except Exception as e:
        print(f"   ❌ Cleanup error: {e}")
    
    # Show final statistics
    print("\n📊 After Cleanup:")
    stats_after = memory.get_memory_stats()
    print(f"   Total interactions: {stats_after.get('total_interactions')}")
    print(f"   Cached interactions: {stats_after.get('cached_interactions')}")
    
    print("\n✨ Cleanup Features:")
    print("   • Configurable age thresholds")
    print("   • Preserves recent interactions")
    print("   • Maintains vector store consistency")
    print("   • Can be scheduled for automatic maintenance")

else:
    print("⚠️ Skipping cleanup demo - memory system not initialized")

## 🎉 Demo Summary and Conclusions

Let's summarize what we've demonstrated and the capabilities of the Gianna Semantic Memory System.

In [ ]:
print("🎉 Gianna Semantic Memory System - Demo Summary")
print("=" * 60)

print("\n✅ Successfully Demonstrated:")

demonstrated_features = [
    "🧠 Semantic embedding and storage of conversations",
    "🔍 Intelligent similarity search with configurable thresholds", 
    "🎯 Automatic clustering of related interactions",
    "📋 Context summarization with intelligent aggregation",
    "🧩 Pattern detection and user preference analysis",
    "🔧 Tool integration for AI agents and workflows",
    "🔄 Seamless integration with existing GiannaState system",
    "⚡ Performance optimization with caching strategies",
    "🧹 Automatic cleanup and maintenance capabilities",
    "🛡️ Fallback strategies for missing dependencies"
]

for feature in demonstrated_features:
    print(f"   {feature}")

print("\n🏗️ System Architecture Highlights:")
architecture_points = [
    "Multiple embedding providers (OpenAI, SentenceTransformers, HuggingFace)",
    "Multiple vector stores (ChromaDB, FAISS, In-Memory)",
    "Intelligent provider selection with automatic fallbacks",
    "Persistent storage with SQLite integration",
    "Configurable similarity thresholds and search parameters",
    "Tool interface compatible with LangChain/LangGraph workflows",
    "Extensible architecture for adding new providers"
]

for point in architecture_points:
    print(f"   • {point}")

if GIANNA_MEMORY_AVAILABLE:
    print("\n📊 Final System Status:")
    if memory:
        final_stats = memory.get_memory_stats()
        print(f"   Configuration: {final_stats.get('embedding_provider')} + {final_stats.get('vector_store_provider')}")
        print(f"   Total interactions processed: {final_stats.get('total_interactions')}")
        print(f"   Embedding dimension: {final_stats.get('embedding_dimension')}")
        print(f"   Cache efficiency: {final_stats.get('cache_hit_rate', 0):.1%}")
    
    available_embeddings = get_available_providers()
    available_vectorstores = get_available_vector_stores()
    print(f"   Available embedding providers: {len(available_embeddings)}")
    print(f"   Available vector stores: {len(available_vectorstores)}")
else:
    print("\n⚠️ Note: Some features were not demonstrated due to missing dependencies")
    print("   Install optional packages for full functionality:")
    print("   • pip install sentence-transformers (local embeddings)")
    print("   • pip install transformers torch (HuggingFace embeddings)")
    print("   • pip install faiss-cpu (FAISS vector store)")
    print("   • pip install chromadb (ChromaDB vector store)")

print("\n🚀 Next Steps:")
next_steps = [
    "Integrate with your Gianna workflows using the tool interface",
    "Configure embedding and vector store providers for your use case",
    "Set up automatic cleanup schedules for production environments",
    "Customize similarity thresholds based on your domain",
    "Extend with custom embedding providers or vector stores as needed"
]

for step in next_steps:
    print(f"   1. {step}")
    next_steps[0] = "2."  # Simple way to increment

print("\n✨ The Gianna Semantic Memory System is ready for production use!")
print("   Built with flexibility, performance, and reliability in mind.")
print("   Enjoy enhanced conversational AI with long-term memory! 🎯")